In [2]:
!pip install jsonlines
!pip install cython
!pip install youtokentome
!ls

  Obtaining dependency information for cython from https://files.pythonhosted.org/packages/f0/89/b1ae45689abecca777f95462781a76e67ff46b55495a481ec5a73a739994/Cython-3.0.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 10.4 MB/s eta 0:00:0000:010:01
  Using cached youtokentome-1.0.6.tar.gz (86 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for youtokentome: filename=youtokentome-1.0.6-cp310-cp310-linux_x86_64.whl size=193318 sha256=cdc6f437e9d42020b0097566f1dc874101ae4167a6b3f300b1cf7e7e0098721b
  Stored in directory: /home/ak/.cache/pip/wheels/df/85/f8/301d2ba45f43f30bed2fe413efa760bc726b8b660ed9c2900c
Successfully built youtokentome
Untitled.ipynb			    test_no_reference  translator.ipynb    val
a-PyTorch-Tutorial-to-Transformers  train	       translator_0.ipynb


In [2]:
!gdown --fuzzy https://drive.google.com/file/d/1izkZacHdzcx4LK8YJsTiRaYp5PEuh3eM/view?usp=drive_link
!gdown --fuzzy https://drive.google.com/file/d/1ugyqEZv36GxTtO0HrAjo54vGmFsmtILI/view?usp=drive_link
!gdown --fuzzy https://drive.google.com/file/d/1zNV4DUd6vhvXTw3HSLDy8Nm6qMSpD7R4/view?usp=drive_link

Downloading...
From: https://drive.google.com/uc?id=1izkZacHdzcx4LK8YJsTiRaYp5PEuh3eM
To: /content/train
100% 44.7M/44.7M [00:00<00:00, 86.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ugyqEZv36GxTtO0HrAjo54vGmFsmtILI
To: /content/test_no_reference
100% 369k/369k [00:00<00:00, 99.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zNV4DUd6vhvXTw3HSLDy8Nm6qMSpD7R4
To: /content/val
100% 262k/262k [00:00<00:00, 80.5MB/s]


In [ ]:


# with jsonlines.open('hw3/out.jsonl', mode='w') as writer:
#     for ds, sr in zip(list_train_dst[:5], list_train_src[:5]):
#         writer.write({"dst":ds, "src":sr})

In [1]:
import jsonlines
import random
import youtokentome
import codecs

In [2]:
def get_count_sent(split):
    c = dict()
    for i, x in enumerate(split):
        c[x] = c.get(x, 0) + 1
    print(len(c))
    return c

In [18]:
X, Y = [], []
max_x_len = 0
max_y_len = 0
with jsonlines.open('train') as reader:
    for obj in reader:
        X.append(obj['src'])
        Y.append(obj['dst'])
        max_x_len = max(max_x_len, len(obj['src']))
        max_y_len = max(max_y_len, len(obj['dst']))

#         print(obj)
#         print(type(obj))
#         break
assert len(X) == len(Y)
print(len(X))
print(f"max_x - {max_x_len}, max_y - {max_y_len}")
count_X = get_count_sent(X)
count_Y = get_count_sent(Y)

300000
max_x - 321, max_y - 395
256655
263733


In [12]:
count_X = get_count_sent(X)
count_Y = get_count_sent(Y)

256655
263733


In [13]:
Xv, Yv = [], []
with jsonlines.open('val') as reader:
    for obj in reader:
        Xv.append(obj['src'])
        Yv.append(obj['dst'])

#         print(obj)
#         print(type(obj))
#         break
assert len(Xv) == len(Yv)
print(len(Xv))
count_Xv = get_count_sent(Xv)
count_Yv = get_count_sent(Yv)

500
500
500


In [14]:
Xt = []
with jsonlines.open('test_no_reference') as reader:
    for obj in reader:
        Xt.append(obj['src'])
print(len(Xt))

1000


In [6]:
def get_top_n(d, n=None):
    l = sorted([(v, k) for k, v in d.items()], reverse=True)
    if n is None:
        n = len(l)
    return l[:n]

In [ ]:
# wrong_sent = ['▶◠◎◠◎ ◎◬?', '◥▢◕◭▦◭◎▵', '◝◗▱◎◗▽◧◓◨◎▵', ]

In [ ]:
# '◝◗▱◎◗▽◧◓◨◎▵' in Xt

False

In [ ]:
# count_X = get_count_sent(X)
# count_Y = get_count_sent(Y)


In [ ]:
# get_top_n(count_X, )

In [ ]:
# get_top_n(count_Y, )

In [ ]:
# used_set = set()
# for i, x in enumerate(X):
#     if x == '▤◠▷◠▫▱◠◈◬▦ ◎◬?' and Y[i] not in used_set:
#         print(Y[i])
#         used_set.add(Y[i])


In [ ]:
# used_set = set()
# for i, y in enumerate(Y):
# #     if y == '_ _' and X[i] not in used_set:
#     if '_' in y and X[i] not in used_set:
#         print(X[i])
#         print(Y[i])
#         used_set.add(X[i])
#         if X[i] in Xt:
#             print("^^^")

In [19]:
def clean_dupes_x_to_y(X, Y, max_count=8):
    new_x, new_y = [], []
    used_counter = dict()
    count_X = get_count_sent(X)
    pairs = [(x, y) for x, y in zip(X, Y)]
    random.shuffle(pairs)
    X = [p[0] for p in pairs]
    Y = [p[1] for p in pairs]
    for i in range(len(X)):
        x = X[i]
        y = Y[i]
        if count_X[x] == 1:
            for _ in range(max_count - 2):
                new_x.append(x)
                new_y.append(y)
        elif count_X[x] == 2:
            for _ in range((max_count - 2) // 2):
                new_x.append(x)
                new_y.append(y)
        elif count_X[x] == 3:
            for _ in range((max_count - 2) // 3):
                new_x.append(x)
                new_y.append(y)    
        elif count_X[x] < max_count:
            new_x.append(x)
            new_y.append(y)
            continue
        elif len(used_counter.get(x, set())) < max_count:
            if y in used_counter.get(x, set()):
                continue
            else:
                used_counter[x] = used_counter.get(x, set())
                used_counter[x].add(y)
                new_x.append(x)
                new_y.append(y)
    print(f"x - {len(new_x)}, y - {len(new_y)}")
    return new_x, new_y

In [20]:
X, Y = clean_dupes_x_to_y(X, Y) #256655
# x - 1539545, y - 1539545

256655
x - 1539545, y - 1539545


In [ ]:
potentially_broken = "*"

In [61]:
# potentially broken = ●

strange_ones = "●*$^_`{}~¡£¤§ª°´¶º¿ÂÄÉÎÝÞßáäåèéëíîïðñóôøúûýþăđţινο​–—‚•‭™─½á“”&“♪♫\\@=+\xa0\xa0\xa0"
print(len(strange_ones))
strange_ones


76


'●*$^_`{}~\x99\x9d\x9e¡£¤§ª°´¶º¿ÂÄÉÎÝÞßáäåèéëíîïðñóôøúûýþăđţινο\u200b–—‚•\u202d™─½á“”&“♪♫\\@=+\xa0\xa0\xa0'

In [ ]:
# for k, v in singles.items():
#     if v == 11:
#         print(f"this is k -{k}-")
#         strange_ones += k
# print(len(strange_ones))
# strange_ones


this is k - -
76


'●*$^_`{}~\x99\x9d\x9e¡£¤§ª°´¶º¿ÂÄÉÎÝÞßáäåèéëíîïðñóôøúûýþăđţινο\u200b–—‚•\u202d™─½á“”&“♪♫\\@=+\xa0\xa0\xa0'

In [ ]:
# for i, x in enumerate(X):
#     for c in x:
# #         if c in strange_ones:
#         if c in potentially_broken:
# #             X[i] = x.replace(c, "")
#             print(x)
#             print(Y[i])
#             break

In [62]:
def print_chars(X):
    tokens = set()
    for x in X:
        tokens.update(set(x))
    print(len(tokens))
    tokens = "".join(sorted(list(tokens)))
    print(tokens)
#      "".join(sorted(list(tokens)))
    return "".join(sorted(list(tokens)))

In [63]:
tokens_X = print_chars(X)
tokens_Xv = print_chars(Xv)
tokens_Xt = print_chars(Xt)
print(set(tokens_X).difference(set(tokens_Xv)))
print(set(tokens_Xv).difference(set(tokens_X)))
print(set(tokens_Xv).difference(set(tokens_Xt)))
print(set(tokens_Xt).difference(set(tokens_Xv)))


176
 !"#$%'()*+/0123456789:;=?@[\]^_`{}~ ¡£¤§ª°´¶º¿ÂÄÉÎÝÞßáâäåèéëíîïðñóôøúûýþăđţινο​–—‚•‭™─■□▢▣▤▥▦▧▨▩▪▫▬▭▮▯▰▱▲△▴▵▶▷▸▹►▻▼▽▾▿◀◁◂◃◄◅◆◇◈◉◊○◌◍◎●◐◑◒◓◔◕◖◗◘◙◚◛◜◝◞◟◠◡◢◣◤◥◦◧◨◩◪◫◬◭◮◯◰◱◲◳♪♫
116
 !"%&'()+/0123456789:;?@[]âéí‘’“”■□▢▣▤▥▦▧▨▩▪▫▬▭▮▯▰▱▲△▴▵▶▷▸▹►▻▼▽▾▿◀◁◂◃◄◅◆◇◈◉◊○◌◍◎◐◑◒◓◔◕◖◗◘◙◚◛◜◝◞◟◠◡◢◣◤◥◦◧◨◩◪◫◬◭◮◯◰◱◲◳
114
 !"#%&'()/0123456789:;?@[]½áâ“”■□▢▣▤▥▦▧▨▩▪▫▬▭▮▯▰▱▲△▴▵▶▷▸▹►▻▼▽▾▿◀◁◂◃◄◅◆◇◈◉◊○◌◍◎◐◑◒◓◔◕◖◗◘◙◚◛◜◝◞◟◠◡◢◣◤◥◦◧◨◩◪◫◬◭◮◯◰◱◲◳
{'´', '`', 'è', '¤', 'ι', 'å', 'ô', 'ä', 'É', 'º', 'ñ', 'ν', 'ο', 'ă', 'î', '}', 'ý', '\xa0', 'Ä', '=', '¡', '\u202d', '–', '*', 'Ý', '—', 'ţ', '♫', 'ë', '§', 'Þ', 'đ', '\u200b', '$', '#', 'ª', 'ø', '¿', '•', '\\', 'ð', 'ß', '●', '{', '─', '°', 'ú', 'û', 'þ', 'Â', '♪', 'á', '\x99', '£', '\x9d', '~', 'Î', 'ó', '\x9e', '‚', '_', '¶', '^', '™', 'ï'}
{'”', '“', '’', '‘', '&'}
{'+', 'í', '’', '‘', 'é'}
{'½', '#', 'á'}


In [64]:
for i, x in enumerate(X):
    for c in strange_ones:
        if c in x and c not in tokens_Xt:
#             print(X[i])
            X[i] = X[i].replace(c, "")
#             print(c)
#             print(X[i])
#             break
#     break
#             print(c)
for i, x in enumerate(Xv):
    for c in strange_ones:
        if c not in tokens_Xt:
            Xv[i] = Xv[i].replace(c, "")
#     for c in x:
#         if c in strange_ones:
#             X[i] = x.replace(c, "")

In [ ]:
# for c in strange_ones:
#     if c not in tokens_Xt:
#         print(c)

In [ ]:
tokens_Xt

' !"#%&\'()/0123456789:;?@[]½áâ“”■□▢▣▤▥▦▧▨▩▪▫▬▭▮▯▰▱▲△▴▵▶▷▸▹►▻▼▽▾▿◀◁◂◃◄◅◆◇◈◉◊○◌◍◎◐◑◒◓◔◕◖◗◘◙◚◛◜◝◞◟◠◡◢◣◤◥◦◧◨◩◪◫◬◭◮◯◰◱◲◳'

In [65]:
counter = 0
for i, y in enumerate(Y):
    for c in bad_y_chars:
        if c in y:
            print(c)
            print(y)
            print(X[i])
            counter += 1
            break
    if counter > 100:
        break


ï
- No parents? I will find my aunt Patience at the Auberge de la Jamaïque.
◡◠◎◠◫▨◠ ▭◠▦◬'▦◠ ◕◗◈◗▽◂◓◨◎ ◉▩▦▨▩ □◠▫◫▴▦▼◪ ▶▴◳▢▴◎ ◂◓◠◈◠▵
Æ
He was one of Team Ægir's officers
○◘◕◫◓ ▶◗◎◗▦◗▦ ◞◨◀◠◳▱◠◓▪▦◈◠▦ ◀◫◓◗
Ü
¢Ü The heart of the night ¢Ü¢Ü
◝◩▲◥◄
ã
It is the most expensive penis of the three, ã220,000, and from where I am sitting, it is hard to see why.
◥◉◭ ◫◉◗▦◈◪ ◪▦ ▻◠▷◠▱▪ ▻▴▦◫◞ ◀◨▵ 220 ◀◗▦ ◞▫▴◓▱◫▦ ◚▴ ◂▫◨◓◈◨◐◨◎ ◳◪◓◈◪▦
Ü
¢Ü There's so much fun here for me and you ¢Ü
"◝◨◓◠◈◠ ◀◪▦◫◎ ◫◉◫▦ ◈▴■ ◞◫▢◫▦ ◫◉◗▦
@
@chickendom40, all right, you have 30 followers.
@▼▷◗▼▨◪▦◈◧◎40▵ ◆◭▢◪▱▵ 30 ▫◠▨◗▻◉◗▦ ◚◠◓▵
ë
I suppose you feel the Brontës do that?
◝◓◧▦▫▴ ◢◠◓◈◪◒▱▴◓'◗▦ ◀◇◳▱◪ ◳◠▻▫▪◐▪▦◠ ◗▦◠▦◬◳◧◓◞◨▦▾▢ ◞◠▦◬◓◬◎?
@
== sync, corrected by elderman == @elder_man
◝◇▱▩◎ "◝◪▫▫◪◓ ○▦◕◪▱◞"
ð
You should give Yaðýz a second thought my girl, you love each other.
▮◪▦ ◒▾ ◤◠◐◬▢ ◗◒◗▦◗ ◀◗◓ ◈◠▷◠ ◈▩◒▩▦ ▨◬▢▪◎▵ ▮◗▢ ◀◗◓◀◫◓◗▦◫▢◗ ◞▴◚◗◳◂◓◞▾▦◨▢▵
¿
This was partly an act of De-Ba'athification by slashing governments, was also ¿? Friedman.
◝◨■ ▷◭

¤
¤ What are you doing here?
◝▾◓◠◈◠ ▦◪ ▽◠▻▪▽◧◓◞▾▦◨▢? ◊◎▻◠◓◠▫◧◓▱◨▨ ▫▴▷▱◗▨▴◈◪▵▵▵
ª
âª all dressed up with nowhere to go âª I used to think that s. And lonely boy
▮▵ ◚◪ ◤◠▱▦▪▢ ◆◪▦◉'◫▦ ◤◨▨◠◓◬ ◙◂◐◨ ◟◠▨◠◞◬'▦◈◠▨◗ ▴▦ ◞◬▨◬▼◬ ◉◗◍▫
ã
So I'm going to put another ã380 on your credit card and there's your two seats.
◢◓▴◈◗ ▨◠◓▫◬▦▪▢◈◠▦ 380 ▻◠◨▦◈ ◈◠▷◠ ◉◪▨◫▻■ ◫▨◫▦▼◗
+
What? A B+, they were loose on their stats.
► ◊◞▫◠▫◗◞▫◗▨▱▴◓◗ ▷◠▫◠▱▪▽◈◬▵
İ
You even argued with İbrahim.
◊◀◓◠▷◫◎'▱▴ ◀◫▱◪ ▨◠◚◕◠ ▴◈◗▽◂◓◞▾▦▵
Τ
Τhis, that fucking hοrrible shit there, what is that?
◂◓◠◈◠▨◗ ▨◂◓▨▾▦◉ ◒▴▽■ ▦▴◈◗◓ ◀◨▦▱◠◓?
ð
Have you talked to the farmers in Svarfaðardalur?
▮◚◠◓◍◠◐◠◓◈◠▱◨◓◈◠▨◗ ◉◫◍▫◉◫▱◪◓▱◪ ▨◧▦▾◒▫◨▦ ◎▾ ▷◫◉?
à
St. Thomas à Becket might have lived to a ripe old age if he...
○▢◫▢ ▶▷◧◎◠◞ ◝◪▼▨▴▫ ◨▢▾▦ ◀◗◓ ◣◎◭◓ ◞▩◓▴◀◫▱◫◓◈◫ ▴◐◪◓▵▵▵
ﬂ
You told me I could right the wrongs of my life by stepping on the right butterﬂies.
◙◧◐◓◨ ◠◈▪◎▱◠◓▱◠ ◗▱▴◓▱◪▽◫▻ ▷◠▽◠▫▪◎◈◠▨◫ ◳◠▦▱◬◒▱◠◓◬ ◈▩▢▴▱▫◎▴◎◗ ◞◪▦ ◞◇◳▱◪◈◗▦▵
ﬁ
We should get married ﬁlst...
► ◲▱▨

In [66]:
tokens_Y = print_chars(Y)
tokens_Yv = print_chars(Yv)


197
 !"#$%'()*+,-./0123456789:;=?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{}~ ¡¢£¤§©ª¬­¯°±³´¶½¿ÀÁÂÃÄÆÉËÐÖÜÝàáâãäåæçèéêëìíïðñòóôõöøúüýþăćēİıœşšŻưΒΗΙΜΝΟΤίνορυсу‒–—‚‭′″€™─│●♪♫檛鈥ﬁﬂ
88
 !"$%&'()+,-./0123456789:;?@ABCDEFGHIJKLMNOPQRSTUVWXYZ\abcdefghijklmnopqrstuvwxyz£éü–—“”


In [67]:
def get_singles_pairs(X):
    singles = dict()
    pairs = dict()
    total_count = 0
    for x in X:
        s = x + '\n'
        for i in range(len(s) - 1):
            p = s[i: i + 2]
            singles[x[i]] = singles.get(x[i], 0) + 1
            pairs[p] = pairs.get(p, 0) + 1
            total_count += 1
    print(total_count)
    return singles, pairs, total_count

In [68]:
singles_X, pairs_X, total_count_X = get_singles_pairs(X)
singles_Xv, pairs_Xv, total_count_Xv = get_singles_pairs(Xv)
singles_Xt, pairs_Xt, total_count_Xt = get_singles_pairs(Xt)


9656995
68368
131486


In [69]:
singles_Y, pairs_Y, total_count_Y = get_singles_pairs(Y)
singles_Yv, pairs_Yv, total_count_Yv = get_singles_pairs(Yv)


10870604
65816


In [70]:
replacers = {
    '´': "'",
    'Ν': "N",
    'Ο': "O",
    'Μ': "M",
    'Β': "B",
    'Η': "H",
#     'ν': "v",
    'Τ': "T",
#     'Ι': "I",

}

In [19]:
# with codecs.open("en.txt", "w", encoding="utf-8") as f:
#     f.write("\n".join(Y))


# with codecs.open("z.txt", "w", encoding="utf-8") as f:
#     f.write("\n".join(X))

In [50]:
bad_y_chars  = "鈥檛●│″′‚усưŻšēćăòìêåäÖÐËÆÄÁ³±°¯©\x97\x90\x8d\x83"
bad_y_chars += "œİõðëãàÝÉÂ"
bad_y_chars += "Ü§¢\x9d@ïίó¿ﬂ\xad¬¡ÃªΤýôçæ¤\x99‒νΗşıþúøÀ^+ﬁ─™€\u202dρ"

In [71]:
for i, y in enumerate(Y):
#     for c in y:
    for k, v in replacers.items():
        Y[i] = Y[i].replace(k, v)
#         print(c, y)
#         break

In [30]:
def skip_chars(A, B, chars):
    new_a = []
    new_b = []
    for i, a in enumerate(A):
#         if len(a) < 3:
#             continue
        for c in chars:
            if c in a:
                break
        else:
            new_a.append(a)
            new_b.append(B[i])
    return new_a, new_b

In [33]:
print(len(Y), len(X))
Y, X = skip_chars(Y, X, "_")
print(len(Y), len(X))

1539545 1539545
1538338 1538338


In [35]:
for i, y in enumerate(Y):
    if len(y) < 2:
        print(X[i])
        print(y)
        

▮◂▦▾◉▱◠◓◬ ◀◂◒ ◚▴◓◫▦▵
?
▮◂▦▾◉▱◠◓◬ ◀◂◒ ◚▴◓◫▦▵
?
▮◂▦▾◉▱◠◓◬ ◀◂◒ ◚▴◓◫▦▵
?
▮◂▦▾◉▱◠◓◬ ◀◂◒ ◚▴◓◫▦▵
?
▮◂▦▾◉▱◠◓◬ ◀◂◒ ◚▴◓◫▦▵
?
▮◂▦▾◉▱◠◓◬ ◀◂◒ ◚▴◓◫▦▵
?
◝▴▦◫ ◣▱◈▩◓◎▴▦◫▢▴ ◫▢◗▦ ◚◪◓◎◪▽▴▼▴◐◗◎▵▵▵
?
◝▴▦◫ ◣▱◈▩◓◎▴▦◫▢▴ ◫▢◗▦ ◚◪◓◎◪▽▴▼▴◐◗◎▵▵▵
?
◝▴▦◫ ◣▱◈▩◓◎▴▦◫▢▴ ◫▢◗▦ ◚◪◓◎◪▽▴▼▴◐◗◎▵▵▵
?
◝▴▦◫ ◣▱◈▩◓◎▴▦◫▢▴ ◫▢◗▦ ◚◪◓◎◪▽▴▼▴◐◗◎▵▵▵
?
◝▴▦◫ ◣▱◈▩◓◎▴▦◫▢▴ ◫▢◗▦ ◚◪◓◎◪▽▴▼▴◐◗◎▵▵▵
?
◝▴▦◫ ◣▱◈▩◓◎▴▦◫▢▴ ◫▢◗▦ ◚◪◓◎◪▽▴▼▴◐◗◎▵▵▵
?
▲◭▫◍◪▦ ◈◇▦!
!
▲◭▫◍◪▦ ◈◇▦!
!
▲◭▫◍◪▦ ◈◇▦!
!
▲◭▫◍◪▦ ◈◇▦!
!
▲◭▫◍◪▦ ◈◇▦!
!
▲◭▫◍◪▦ ◈◇▦!
!
◝◧◒ ◎▾?
!
◅◪◚◗◓◫: ◎◪◓▫▦
!
◅◪◚◗◓◫: ◎◪◓▫▦
!
◅◪◚◗◓◫: ◎◪◓▫▦
!
◅◪◚◗◓◫: ◎◪◓▫▦
!
◅◪◚◗◓◫: ◎◪◓▫▦
!
◅◪◚◗◓◫: ◎◪◓▫▦
!
◛◉◠▨ ◠▱◎◠▨ ◎◬?
!
◛◉◠▨ ◠▱◎◠▨ ◎◬?
!
◛◉◠▨ ◠▱◎◠▨ ◎◬?
!
◛◉◠▨ ◠▱◎◠▨ ◎◬?
!
◛◉◠▨ ◠▱◎◠▨ ◎◬?
!
◛◉◠▨ ◠▱◎◠▨ ◎◬?
!
►◊▱◠◉ ◚◪◓◎◪◳◪ ◎◗?
!
►◊▱◠◉ ◚◪◓◎◪◳◪ ◎◗?
!
►◊▱◠◉ ◚◪◓◎◪◳◪ ◎◗?
!
►◊▱◠◉ ◚◪◓◎◪◳◪ ◎◗?
!
►◊▱◠◉ ◚◪◓◎◪◳◪ ◎◗?
!
►◊▱◠◉ ◚◪◓◎◪◳◪ ◎◗?
!
◝▩▽▩▨ ▨◨▨◨▱▾ ▨◬▢▱◠◓◈◠▦ ▷◂◒▱◠▦▪◳◧◓▾◎▵)
?
◝▩▽▩▨ ▨◨▨◨▱▾ ▨◬▢▱◠◓◈◠▦ ▷◂◒▱◠▦▪◳◧◓▾◎▵)
?
◝▩▽▩▨ ▨◨▨◨▱▾ ▨◬▢▱◠◓◈◠▦ ▷◂◒▱◠▦▪◳◧◓▾◎▵)
?
◝▩▽▩▨ ▨◨▨◨▱▾ ▨◬▢▱◠◓◈◠▦ ▷◂◒▱◠▦▪◳◧◓▾◎▵)
?
◝▩▽▩▨ ▨◨▨◨▱▾ ▨◬▢▱◠◓◈◠▦ ▷◂◒▱◠▦▪◳◧◓▾◎▵)
?
◝▩▽▩▨ ▨◨▨◨

In [36]:
with open("train.zeex", "w") as f:
    f.write("\n".join(X))
with open("train.eneex", "w") as f:
    f.write("\n".join(Y))
with open("val.z", "w") as f:
    f.write("\n".join(Xv))
with open("val.en", "w") as f:
    f.write("\n".join(Yv))    
with open("test.z", "w") as f:
    f.write("\n".join(Xt))

In [10]:
a = X.copy()
a.extend(Y)
a.extend(Xv)
a.extend(Yv)
a.extend(Xt)
with open("all_text", "w") as f:
    f.write("\n".join(a))

In [39]:
a = X.copy()
a.extend(Xv)
with open("train.zveex", "w") as f:
    f.write("\n".join(a))

In [40]:
a = Y.copy()
a.extend(Yv)
with open("train.enveex", "w") as f:
    f.write("\n".join(a))

In [36]:
Xt[0]

'◲▦◠▦◬▦■ ◉◗▢◕◗ ◍◗▱◎ ▽◠▽▪▦◠ ◕▴◉◗▦▼▴ ◀◗◓◉◧▨ ◎▴◞◠▸ ◠▱◈▪▨ ◚◪ ◀◨ ◎◪◞◠▸▱◠◓◬▦ ◀◠▢▪▱◠◓▪ ▻◪▨ ◈◂◞▫◉◠ ◈▴◐◫▱◈◗▵'

In [26]:
sum([len(y) for y in Y])/len(Y)

41.61123708628199

In [27]:
"".join([x[1] for x in get_top_n(singles_Y)])

' etoanishrlud.ymgwcf,\'pbkIv?-TWYASHMNjB!ODCLxGE"PRFJ0zKUq1V2:534♪9687)[](ZQ#$*_;X/%é~`\xa0¶ο=}–ö{—\\♫íüυ\x92£ñèáâ½'

In [ ]:
get_top_n(singles_Y)

[(1847364, ' '),
 (966876, 'e'),
 (746470, 't'),
 (723517, 'o'),
 (607000, 'a'),
 (532734, 'n'),
 (484529, 'i'),
 (460680, 's'),
 (457347, 'h'),
 (427687, 'r'),
 (333839, 'l'),
 (305954, 'u'),
 (282142, 'd'),
 (266903, '.'),
 (239370, 'y'),
 (206986, 'm'),
 (190186, 'g'),
 (188608, 'w'),
 (168842, 'c'),
 (139315, 'f'),
 (122218, ','),
 (121267, "'"),
 (117104, 'p'),
 (113867, 'b'),
 (110326, 'k'),
 (106561, 'I'),
 (82084, 'v'),
 (50756, '?'),
 (40178, '-'),
 (39396, 'T'),
 (34607, 'W'),
 (28612, 'Y'),
 (27298, 'A'),
 (25842, 'S'),
 (22046, 'H'),
 (16313, 'M'),
 (15619, 'N'),
 (15054, 'j'),
 (14907, 'B'),
 (14797, '!'),
 (14350, 'O'),
 (14316, 'D'),
 (13621, 'C'),
 (11793, 'L'),
 (10121, 'x'),
 (9866, 'G'),
 (9825, 'E'),
 (8996, '"'),
 (8017, 'P'),
 (7825, 'R'),
 (6822, 'F'),
 (6438, 'J'),
 (5172, '0'),
 (4958, 'z'),
 (4165, 'K'),
 (4096, 'U'),
 (3697, 'q'),
 (2995, '1'),
 (2355, 'V'),
 (2001, '2'),
 (1562, ':'),
 (1378, '5'),
 (1195, '3'),
 (1046, '4'),
 (1041, '♪'),
 (870, '9'),
 (702

In [ ]:
get_top_n(pairs_X)

[(184453, '▵\n'),
 (163395, '▦ '),
 (142613, ' ◀'),
 (128592, '◠ '),
 (122747, '◠◓'),
 (114839, '◠▦'),
 (107941, '▱◠'),
 (101542, '◓ '),
 (84365, '◎◠'),
 (81772, ' ◈'),
 (79300, '◈◠'),
 (78804, ' ▨'),
 (69718, ' ◞'),
 (69465, ' ◕'),
 (66465, '■ '),
 (65686, '▨ '),
 (63521, '◪ '),
 (63379, '▴ '),
 (61165, ' ◠'),
 (59800, '◓◠'),
 (59040, '▨◠'),
 (56331, '◠▨'),
 (56054, '◗▦'),
 (55845, '◠▱'),
 (55655, '◫▦'),
 (54748, '◪▦'),
 (54590, '▴▦'),
 (54588, '◗ '),
 (54169, '◠◎'),
 (54086, '◫ '),
 (52580, '▴◓'),
 (52550, '◪◓'),
 (51660, '▦◈'),
 (48270, '◎ '),
 (46958, '◎▵'),
 (44595, '◀◫'),
 (44441, '◫◓'),
 (44341, '◀◗'),
 (44224, '▱◪'),
 (44161, '◈◪'),
 (44115, '▱▴'),
 (44112, '◈▴'),
 (44081, '◗◓'),
 (43707, '◠◈'),
 (43514, '?\n'),
 (41028, ' ◎'),
 (39290, '◂◓'),
 (38999, '◧◓'),
 (38125, '▦◠'),
 (37645, '▪▦'),
 (37594, '◬▦'),
 (37123, ' ▷'),
 (37023, '◞◠'),
 (35700, '◓▵'),
 (35447, ' ◚'),
 (35344, '▽◠'),
 (35229, '◳◠'),
 (34800, '▫◠'),
 (33726, '◫▱'),
 (33119, '◗▱'),
 (33041, '◀◠'),
 (32953, '◓◈')

In [ ]:
len(pairs_X)

2278

In [ ]:
X[101000:101020]

['◰◞▫◂▦▽◠ ◉◧▨ ◨▢◠▨▫◠▽◈▪▵',
 '◝▾ ◠◓◠◈◠ ◠▽◠◐▪▦◬ ▦◠◞◬▱ ◳◠◓◠▱◠◈▪▦?',
 '► ▭▴▻ ◧▱◨◓ ◀◇▽▱◪ ◒◪◳▱▴◓▵',
 '◆◪◓◫ ◈◇▦▩▦!',
 '◝▾ ◀◫◓ ◞◪◉▴▦▴▨▵',
 '◳◠◓◠ ▨◠◀◨▨▱◠◓◬ ◞◭◓▴▨▱◫ ▨◨◞◎◠■ ◠▨▼◫◐▴◓▱◪◓◈◪ ◞▪◚◬ ◀◗◓◗▨◫◎◫■ ◈◠◎◠◓▱◠◓◬▦ ▨◠▻◠▦◎◠◞◬▵',
 '► ▭◠◳◬◓■ ◓▩◳◠▱◠▦◎◠◈▪◎▵',
 '◦▱▨▪◒ ▱▩▫◍◪▦!',
 '► ◙◠▷◠ ◎◬ ▨◇▫▩?',
 '◝◨◓▦◨◎▾ ◠▱▫▪▦ ◀◫◓ ▨◭▻◪ ◫▱▴ ◈◭▢▱▴◒▫◗◓◈◗▦◫▢▵',
 '○◓▫▪▨ ◀▾◓◠◈◠▦ ◕◫▫◞▴▨■ ▦◪ ◈▴◓◞◗▦◗▢ ▷◠?',
 '◦◗▱◪◎ ◀◪▦◫ ◀◪▨▱◫◳◧◓▵',
 '▭▴▽ ▷◠◈◗ ◦◎▴◓◫▨◠▦ ▫▴▱◪◚◫▢◳◧▦◨ ◫▢▱◪◳◪▱◫◎▵',
 '► ◄◨▷▫◪◒▴◎■ ◎▾▷▫▴◒◪◎▵',
 '▮▫◠▦◈◬ ◈◗◐▴◓ ▫◠◓◠◍▫◠▵',
 "▿ ◳◭▢◈◪▦ □◠◎◪▱◠'▦◬▦ ▷◗◞◞◪▱◪◓◫▦◪",
 '► ◰◠◓▱◪ ◞▴▦◈▴▦ ▦▴ ◗◞▫◗◳◂◓?',
 '◝◠◒▨◠ ◀◫◓ ▴◚ ◠▱◠▼◠▨ ▻◠◓◠◎◬▢ ◧▱◎◠◳◠▼◠▨▵',
 '◱ ▨◠◈◠◓ ◗◉◎◪▵',
 '◀◗◓ ◈◠▨◫▨◠▦ ◚◠◓ ◎▪?']

In [ ]:
used_keys = []
for k in singles_X.keys():
    train_q = singles_X.get(k, 0)
    test_q = singles_Xt.get(k, 0)
    print(k, train_q, test_q)
    used_keys.append(k)
print('===')
for k in singles_Xt.keys():
    if k not in used_keys:
        train_q = singles_X.get(k, 0)
        test_q = singles_Xt.get(k, 0)
        print(k, train_q, test_q)



○ 11441 160
▦ 589994 8808
▴ 358945 5071
◎ 375204 3318
  1128877 16085
◇ 30409 420
▱ 430206 7026
◈ 330328 4750
▩ 62607 884
◓ 526713 7241
◭ 63485 856
◐ 90781 1106
◪ 359698 5110
◂ 117602 1719
◨ 141517 1822
◳ 148685 1698
◗ 357601 4611
◕ 92277 1484
◉ 86685 1207
▨ 314100 4564
▫ 226508 3722
◀ 196644 1975
◠ 880605 12700
◒ 126939 1509
◫ 357611 4702
▽ 148326 1798
▻ 57011 1022
▪ 184961 2526
◬ 184922 2540
▢ 131537 1282
▵ 235242 1052
◱ 7650 36
▼ 78322 1167
◞ 237011 3631
▮ 27761 273
◧ 116968 1599
▾ 140984 1724
■ 69089 895
▶ 12555 205
▯ 12294 113
► 36478 52
3 1257 63
◍ 29635 533
▤ 4226 144
▷ 74624 1137
◝ 58971 477
◣ 30529 434
▸ 2972 90
◙ 13110 252
â 2828 17
□ 7671 224
◤ 5850 50
◚ 69723 1117
' 20962 798
? 46210 27
◑ 5121 55
◢ 17913 232
# 869 1
▭ 19288 124
◊ 5300 55
◥ 815 12
▬ 5694 184
◰ 6280 64
◅ 6791 46
▰ 1522 89
◦ 11607 157
! 13328 2
◄ 8856 233
▥ 2058 83
" 7994 510
▿ 7690 37
◘ 6313 57
1 2844 161
◆ 11251 134
6 724 43
0 4272 234
▲ 4600 116
◩ 1742 7
◲ 5155 44
( 168 27
◃ 944 22
◯ 967 26
) 170 27
◟ 5799 

In [ ]:
# for c in strange_ones:
#     print(c, singles.get(c, 0))

In [ ]:
used_keys = []
for k in pairs.keys():
    train_q = pairs.get(k, 0)
    test_q = pairs_t.get(k, 0)
    print(k, train_q, test_q, train_q / total_count, test_q / total_count_t)
    used_keys.append(k)
print('===')
for k in pairs_t.keys():
    if k not in used_keys:
        train_q = pairs.get(k, 0)
        test_q = pairs_t.get(k, 0)
        print(k, train_q, test_q, train_q / total_count, test_q / total_count_t)



◄▴ 886 27 8.854136790617253e-05 0.00020534505574737994
▴◓ 53780 839 0.005374441045139908 0.006380907473038955
◓◠ 61151 876 0.006111053260530876 0.006662306253137216
◠▨ 57285 854 0.005724709097635546 0.006494988059565277
▨  66668 1006 0.006662388166556107 0.007651004669698675
 ◨ 4401 73 0.0004398087586400286 0.0005551921877614347
◨▽ 3575 38 0.00035726342016316795 0.00028900415253334954
▽◠ 35797 460 0.003577331091351307 0.0034984713201405473
◠▦ 117352 1769 0.01172743409314352 0.01345390383767093
▦◈ 52125 1090 0.005209050566714722 0.00828985595424608
◈◬ 19661 243 0.001964798910161691 0.0018481055017264196
◬◓ 16026 142 0.0016015394605692112 0.0010799628857825168
◓▪ 18979 314 0.001896644042315179 0.002388086944617678
▪▼ 1220 19 0.0001219192650626755 0.00014450207626667477
▼◬ 3193 36 0.00031908869946321546 0.0002737934076631733
◬▵ 6399 87 0.0006394765386361151 0.0006616674018526688
▵ 199941 839 0.019980868668767544 0.006380907473038955
▽◪ 7731 129 0.0007725883919668396 0.000981093044126371
◪

In [ ]:
singles

{'◲': 44,
 '▦': 8808,
 '◠': 12700,
 '◬': 2540,
 '■': 895,
 ' ': 16085,
 '◉': 1207,
 '◗': 4611,
 '▢': 1282,
 '◕': 1484,
 '◍': 533,
 '▱': 7026,
 '◎': 3318,
 '▽': 1798,
 '▪': 2526,
 '▴': 5071,
 '▼': 1167,
 '◀': 1975,
 '◓': 7241,
 '◧': 1599,
 '▨': 4564,
 '◞': 3631,
 '▸': 90,
 '◈': 4750,
 '◚': 1117,
 '◪': 5110,
 '◨': 1822,
 '▻': 1022,
 '◂': 1719,
 '▫': 3722,
 '◐': 1106,
 '◫': 4702,
 '▵': 1052,
 '▯': 113,
 '▥': 83,
 '◟': 58,
 '◳': 1698,
 '◒': 1509,
 '◣': 434,
 '"': 510,
 '▷': 1137,
 '▾': 1724,
 '◡': 78,
 '◝': 477,
 '▹': 28,
 '◭': 856,
 '◇': 420,
 '▰': 89,
 '▮': 273,
 '◆': 134,
 "'": 798,
 '○': 160,
 '▩': 884,
 '▭': 124,
 '◄': 233,
 '◢': 232,
 '1': 161,
 '◰': 64,
 '▤': 144,
 '▬': 184,
 '▲': 116,
 '(': 27,
 '/': 6,
 ')': 27,
 '◦': 157,
 '◙': 252,
 '□': 224,
 '2': 140,
 '9': 47,
 '◘': 57,
 '▶': 205,
 ':': 85,
 '◌': 100,
 '◱': 36,
 '▧': 10,
 '◅': 46,
 '◖': 23,
 '6': 43,
 '4': 49,
 '?': 27,
 '3': 63,
 '0': 234,
 '◩': 7,
 '8': 41,
 '◑': 55,
 '▣': 4,
 '◊': 55,
 '◤': 50,
 ';': 7,
 '◥': 12,
 '◯': 26,